In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time


In [2]:
SEASONS = list(range(2016,2023))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR= os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")



In [4]:
async def get_html(url, selector, sleep=15, retries=1):
    html = None
    for i in range(1, retries+1):
        time.sleep((sleep * i) * 5)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout Error on {url}")
            continue
        else:
            break
    return html


In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
###for season in SEASONS:
    ###await scrape_season(season)

In [7]:
standings_files = os.listdir(STANDINGS_DIR)

In [8]:
standings_files = os.path.join(STANDINGS_DIR, standings_files[0])
with open(standings_files, 'r') as f:
    html = f.read()
soup = BeautifulSoup(html)
link2 = soup.find_all('a')

In [9]:
link2

[]